## Inteligencia Artificial para la Ciencia de los Datos
## Máster en Lógica, Computación e Inteligencia Artificial 
## Tema 2: Modelos probabilísticos

In [1]:
import numpy as np    

### Ejercicio 2: clasificación de textos usando Naive Bayes multinomial

Este ejemplo está parcialmente tomado del libro:

[*Introduction to Machine Learning with Python*](http://shop.oreilly.com/product/0636920030515.do)  
**Andreas C. Müller & Sarah Guido**  
O'Reilly 2017

En concreto, este tema está basado en el capítulo 7, pero con modificaciones. 

Github con el material del libro: [Github](https://github.com/amueller/introduction_to_ml_with_python). 

El libro está accesible *online* desde la [Biblioteca de la Universidad de Sevilla](https://fama.us.es), como recurso electrónico.

### Aplicación: análisis de sentimientos en textos

Como datos en nuestar aplicación usaremos críiticas de películas en la web IMDB (Internet Movie Database). Son críticas que ya vienen con la etiqueta "pos" o "neg", de acuerdo a la puntuación que acompaña a la crítica (positiva, 7 o más; negativa 4 o menos). El objetivo es ser capaz de declarar como positiva o negativa una crítica (por supuesto, sin saber la puntuación que la acompaña).

Los datos están disponibles en http://ai.stanford.edu/~amaas/data/sentiment/, pero usaremos un subconjunto de ellos que se puede obtener descomprimiendo el archivo `aclImdb.tar.gz`.

Al descomprimir tendremos dos carpetas, test y train,en cada una de ellas con dos subcarpetas pos y neg, separando las críticas negativas y positivas, cada una en un archivo individual. 

#### Apartado 1
La función `load_files` que viene en el módulo `datasets` de scikit-learn permite cargar conjuntos de datos que vienen en carpetas con esa estructura. Consultar en el manual para saber cómo usarla para cargar las críticas y sus valores de clasificación en cuatro variables:

* `text_train` y `text_test` ambas listas de strings, y cada elelemnto de esas listas siendo el texto de una revisión. 

* `y_train` e `y_test` con los correspondientes valores de clasificación.


**Nota**: los textos originales tienen muchas etiquetas de cambio de línea que es conveniente quitar. Esto se puede hacer fácilmente así:

```python
[doc.replace(b"<br />", b" ") for doc in textos]
```

Explorar los datos brevemente: cuántos ejemplos hay, cuántos de cada clase, mostrar algunas críticas positivas y negativas,...


In [2]:
# Solución:
import sklearn
from sklearn.datasets import load_files
reviews_train = load_files("aclImdb/train/")
text_train, y_train = reviews_train.data, reviews_train.target
text_train = [doc.replace(b"<br />", b" ") for doc in text_train]

print("Clases: {}".format(reviews_train.target_names))

reviews_test = load_files("aclImdb/test/")
text_test, y_test = reviews_test.data, reviews_test.target

print("Número de ejemplos en test: {}".format(len(text_test)))
print("Ejemplos por cada clase: {}".format(np.bincount(y_test)))

text_test = [doc.replace(b"<br />", b" ") for doc in text_test]

Clases: ['neg', 'pos']
Número de ejemplos en test: 25000
Ejemplos por cada clase: [12500 12500]


### El modelo vectorial *Bag of Words*

Antes de poder aplicar modelos de aprendizaje a textos, debemos representar los documentos mediante vectores numéricos. La forma más fácil de hacerlo es, una vez fijado los términos de nuestro *Vocabulario* (y un orden implícito entre los términos), mediante un vector en el que en cada componente tenemos el número de veces que aparece el correspondiente término del vocabulario, en el documento. Esto se puede hacer fácilmente en scikit learn con `CountVectorizer`.

#### Apartado 2

Para prácticar previamente con ejemplo sencillo, vamos a usar `CountVectorizer` para la vectorización de las siguientes cuatro frases:

In [3]:
cuatro_frases =["Cargamento de oro dañado por el fuego",
              "La entrega de la plata llegó en el camión color plata",
              "El cargamento de oro llegó en un camión",
              "Oro, oro, oro: gritó al ver el camión"]

Se pide:

* Vectorizar estas cuatro frases con `CountVectorizer` (ver detalles en el manual)
* Consultar el vocabulario creado
* Consultar los vectores creados, comprendiendo la representación

*Nota*: puesto que en una representación vectorial de un texto la mayoría de las componentes son cero (todos los términos del vocabulario que no están en el documento), la representación más adecuada es mediante matrices dispersas de Scipy. El método `toarray` nos permite ver las matrices dispersas como arrays de numpy. 


In [4]:
# Solución
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(cuatro_frases)
print(vectorizer.get_feature_names())
print(X.toarray())
# De esta forma representmaos vectorialmente las frases por las repeticiones de las mismas
# El peso de las palabras
# Se convierte en un vector de caracteristicas

vect2 = CountVectorizer(min_df=100, stop_words="english").fit(text_train)
X2_train = vect2.transform(text_train)



['al', 'camión', 'cargamento', 'color', 'dañado', 'de', 'el', 'en', 'entrega', 'fuego', 'gritó', 'la', 'llegó', 'oro', 'plata', 'por', 'un', 'ver']
[[0 0 1 0 1 1 1 0 0 1 0 0 0 1 0 1 0 0]
 [0 1 0 1 0 1 1 1 1 0 0 2 1 0 2 0 0 0]
 [0 1 1 0 0 1 1 1 0 0 0 0 1 1 0 0 1 0]
 [1 1 0 0 0 0 1 0 0 0 1 0 0 3 0 0 0 1]]


#### Apartado 3

Aplicar la vectorización a las críticas de cine, tanto las de entenamiento como las de test, almacenando los datos generados en variables `X_train` y `X_test`. Explorar también el vocabulario que se ha generado (tamaño, algunos términos, etc...)  

In [5]:
# Solución:
vect = CountVectorizer().fit(text_train)
X_train = vect.transform(text_train)
print("X_train:\n{}".format(repr(X_train)))

vectorizer

# en el entrenamiento obtendremos las probabilidades de cada palabra 
# despues deberemos sumarlas

X_train:
<25000x74849 sparse matrix of type '<class 'numpy.int64'>'
	with 3431196 stored elements in Compressed Sparse Row format>


CountVectorizer()

In [6]:
feature_names = vect.get_feature_names()
print("Número de términos en el vocabulario: {}".format(len(feature_names)))
print("Primeras 20 características (términos):\n{}".format(feature_names[:20]))
print("Términos del 20010 al 20030:\n{}".format(feature_names[20010:20030]))
print("Términos cada 2000 posiciones:\n{}".format(feature_names[::2000]))

Número de términos en el vocabulario: 74849
Primeras 20 características (términos):
['00', '000', '0000000000001', '00001', '00015', '000s', '001', '003830', '006', '007', '0079', '0080', '0083', '0093638', '00am', '00pm', '00s', '01', '01pm', '02']
Términos del 20010 al 20030:
['dratted', 'draub', 'draught', 'draughts', 'draughtswoman', 'draw', 'drawback', 'drawbacks', 'drawer', 'drawers', 'drawing', 'drawings', 'drawl', 'drawled', 'drawling', 'drawn', 'draws', 'draza', 'dre', 'drea']
Términos cada 2000 posiciones:
['00', 'aesir', 'aquarian', 'barking', 'blustering', 'bête', 'chicanery', 'condensing', 'cunning', 'detox', 'draper', 'enshrined', 'favorit', 'freezer', 'goldman', 'hasan', 'huitieme', 'intelligible', 'kantrowitz', 'lawful', 'maars', 'megalunged', 'mostey', 'norrland', 'padilla', 'pincher', 'promisingly', 'receptionist', 'rivals', 'schnaas', 'shunning', 'sparse', 'subset', 'temptations', 'treatises', 'unproven', 'walkman', 'xylophonist']


#### Apartado 4

Aplicar a los datos vectorizados el clasificador `MultinomialNB` para obtener predicciones sobre el "sentimiento" de una crítica. Mostrar varios ejemplos de predicciones sobre críticas, tanto del conjunto de entrenamiento como del conjunto de test. Mostrar también el rendimiento global sobre ambos conjuntos, probando también con distintos valores de la constante de suavizado. 

Examinar los atributos `class_count_`, `class_log_prior_`, `feature_count_` y `feature_log_prob_`, entendiendo qué contiene cada uno de ellos. 

In [7]:
(X_train[y_train==0][:,0].toarray()).sum()
# ver uqe hace esta linea
# obtiene el 51 de abajo osea las repeticiones de la frase "0" son 51

51

In [8]:
# Solución
from sklearn.naive_bayes import MultinomialNB
# Regresión logística con el parámetro por defecto
multinb=MultinomialNB().fit(X_train,y_train)

print(multinb.class_count_)

print(multinb.class_log_prior_)

print(multinb.feature_count_)

print(multinb.feature_log_prob_)


[12500. 12500.]
[-0.69314718 -0.69314718]
[[ 51. 174.   1. ...   0.   3.   1.]
 [ 42. 126.   0. ...   1.   1.   0.]]
[[-10.90247427  -9.68893202 -14.16057081 ... -14.85371799 -13.46742363
  -14.16057081]
 [-11.11979709 -10.03681011 -14.8809972  ... -14.18785002 -14.18785002
  -14.8809972 ]]


#### Apartado 5

Los *stop words* son palabras de uso tan frecuente que no aportan nada a la clasificación de textos (ya que no dan información sobre la clase a la que se pertenece). Igualmente, aquellos términos de muy baja frecuencia podrían ignorarse y así ganar en eficiencia (se tendrían menos características). Las opciones `min_df` y `stop_words` del vectorizador nos permiten llevar a cabo esto. 

Se pide vectorizar con ambas opciones (por ejemplo, `min_df=100` y `stop_words=english`), comprobar tamaño del vocabulario y los vectores generados, y el rendimiento del clasificador obtenido de nuevo con `MultinomialNB`. 

In [9]:
# Solución:
print("Segunda crítica del conjunto de test: \n\n{}\n".format(text_test[1]))
print("Clasificación verdadera: {}.\n\n".format(y_test[1]))
print("Tercera crítica del conjunto de test: \n\n{}\n".format(text_test[2]))
print("Clasificación verdadera: {}".format(y_test[2]))
# Sabemos loq ue vale cada critica porque lo tenemos
# ahora veremos con predict que tal

Segunda crítica del conjunto de test: 

b'I don\'t know how this movie has received so many positive comments. One can call it "artistic" and "beautifully filmed", but those things don\'t make up for the empty plot that was filled with sexual innuendos. I wish I had not wasted my time to watch this movie. Rather than being biographical, it was a poor excuse for promoting strange and lewd behavior. It was just another Hollywood attempt to convince us that that kind of life is normal and OK. From the very beginning I asked my self what was the point of this movie,and I continued watching, hoping that it would change and was quite disappointed that it continued in the same vein. I am so glad I did not spend the money to see this in a theater!'

Clasificación verdadera: 0.


Tercera crítica del conjunto de test: 

b"I caught this movie on the Horror Channel and was quite impressed by the film's Gothic atmosphere and tone. As a big fan of all things vampire related, I am always happy to see

In [10]:
print("Predicción del clasificador para la segunda crítica: {}\n".format(multinb.predict(vect.transform([text_test[1]]))[0]))
print("Predicción del clasificador para la tercera crítica: {}".format(multinb.predict(vect.transform([text_test[2]]))[0]))

Predicción del clasificador para la segunda crítica: 0

Predicción del clasificador para la tercera crítica: 1


In [11]:
print("Predicción de probabilidad para la segunda crítica: {}\n".format(multinb.predict_proba(vect.transform([text_test[1]]))[0]))
print("Predicción de probabilidad para la tercera crítica: {}".format(multinb.predict_proba(vect.transform([text_test[2]]))[0]))

Predicción de probabilidad para la segunda crítica: [9.99999862e-01 1.37575474e-07]

Predicción de probabilidad para la tercera crítica: [0.01977596 0.98022404]


In [12]:
print("Primera crítica del conjunto de test: \n\n{}\n".format(text_test[0]))
print("Clasificación verdadera: {}.\n".format(y_test[0]))
print("Predicción del clasificador para la primera crítica: {}\n".format(multinb.predict(vect.transform([text_test[0]]))[0]))
print("Predicción de probabilidad para la primera crítica: {}".format(multinb.predict_proba(vect.transform([text_test[0]]))[0]))

Primera crítica del conjunto de test: 

b"Don't hate Heather Graham because she's beautiful, hate her because she's fun to watch in this movie. Like the hip clothing and funky surroundings, the actors in this flick work well together. Casey Affleck is hysterical and Heather Graham literally lights up the screen. The minor characters - Goran Visnjic {sigh} and Patricia Velazquez are as TALENTED as they are gorgeous. Congratulations Miramax & Director Lisa Krueger!"

Clasificación verdadera: 1.

Predicción del clasificador para la primera crítica: 0

Predicción de probabilidad para la primera crítica: [0.68716538 0.31283462]


In [13]:
X_test = vect.transform(text_test)
print("Rendimiento de multinb sobre el conjunto de entrenamiento: {:.2f}".format(multinb.score(X_train,y_train)))
print("Rendimiento de multinb sobre el conjunto de test: {:.2f}".format(multinb.score(X_test,y_test)))

Rendimiento de multinb sobre el conjunto de entrenamiento: 0.90
Rendimiento de multinb sobre el conjunto de test: 0.81


In [14]:
# Mejorando el entrenamiento con los cambios de features y reduccion de palabras se obtiene las siguiente celdas interesantes
# Hay que saberlas explicar !!!
print("Número de términos en el vocabulario original: {}".format(len(feature_names)))
feature_names2 = vect2.get_feature_names()
print("Número de términos en el vocabulario con stop words y min_df: {}".format(len(feature_names2)))

Número de términos en el vocabulario original: 74849
Número de términos en el vocabulario con stop words y min_df: 3561


In [15]:
multinb2=MultinomialNB(alpha=1).fit(X2_train,y_train)

In [16]:
X2_test = vect2.transform(text_test)
print("Rendimiento de multinb2 sobre el conjunto de entrenamiento {:.2f}".format(multinb2.score(X2_train,y_train)))
print("Rendimiento de multinb2 sobre el conjunto de test: {:.2f}".format(multinb2.score(X2_test,y_test)))

Rendimiento de multinb2 sobre el conjunto de entrenamiento 0.86
Rendimiento de multinb2 sobre el conjunto de test: 0.84
